In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/11-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
# samples = ["Signal", "ccbar"]
samples = ["Signal","Signal_Ds*","All","All_Ds*"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "1126"
Attempt = "0"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
DataFrames[samples[1]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Ds*tree",library='pd')
# Background
DataFrames[samples[2]] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"24_"+ Attempt +"_All.root:Dstree",library='pd')
DataFrames[samples[3]] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"24_"+ Attempt +"_All.root:Ds*tree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["Signal"].columns.tolist()

In [ ]:
DataFrames[samples[1]].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["All"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_rank']==1]

# D0 Invariant Mass
#-----------------------
DataFrames[samples[0]] = DataFrames[samples[0]][(DataFrames[samples[0]]['Ds_D0_sideband']==1)]
DataFrames[samples[2]] = DataFrames[samples[2]][(DataFrames[samples[2]]['Ds_D0_sideband']==1)]

# KaonID
#-----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

In [ ]:
Stacked = False
Density = False
Bins = 30
Range = [0, 30]
Hits = 0
Op = 0.58
var = '__ncandidates__'
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{*+})=1$'
label2= r'$isSignal(D_s^{*+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1][var],
        DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==0][var],
        DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal'].isna()][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# Title
#---------
# Signal
plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

# Best Candidate Selection

In [ ]:
# print(DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==1]['Ds_electron_pt_rank'].value_counts(dropna=False))
# print(DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==1]['Ds_chiProb_Ds_rank'].value_counts(dropna=False))
# print(DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==1]['Ds_random_rank'].value_counts(dropna=False))

In [ ]:
len(DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==1])

In [ ]:
print(DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1]['Ds_star_DeltaM_diff_rank'].value_counts(dropna=False))
print(DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1]['Ds_star_dM_rank'].value_counts(dropna=False))
print(DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1]['Ds_star_chiProb_Ds_star_rank'].value_counts(dropna=False))
# print(DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1]['Ds_star_chiProb_Ds_star_rank'].value_counts(dropna=False))

In [ ]:
len(DataFrames[samples[1]])